# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
#from api_keys import g_key
# this doesnt work for some reason - I "hard coded" my gKEY below, to get the code to work, but then removed before I pushed to Git Hub 

# Import API key
#from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Pull in data from WeatherPy
csv_file = os.path.join('..', 'WeatherPy', 'Output_CSV', 'weather_data.csv')

city_df = pd.read_csv(csv_file)
city_df


,Unnamed: 0,CITY,CLOUDINESS,COUNTRY,DATE,HUMIDITY,LAT,LNG,MAX TEMP,WIND SPEED
0,0,Mataura,100,NZ,1619465169,88,-46.1927,168.8643,39.99,1.01
1,1,Castaños,100,MX,1619465169,14,26.7833,-101.4167,95.72,7.09
2,2,Mar del Plata,0,AR,1619465029,67,-38.0023,-57.5575,62.60,10.36
3,3,Cape Town,75,ZA,1619465160,72,-33.9258,18.4232,64.99,16.11
4,4,Pundaguitan,100,PH,1619465169,75,6.3711,126.1689,80.80,5.84
...,...,...,...,...,...,...,...,...,...,...
1093,1093,Shīeli,6,KZ,1619465294,23,44.1667,66.7500,65.71,6.69
1094,1094,Klaksvík,75,FO,1619465294,87,62.2266,-6.5890,41.00,19.57
1095,1095,Xining,54,CN,1619465294,49,36.6167,101.7667,39.70,3.47
1096,1096,Jennings,75,US,1619465295,36,38.7192,-90.2604,80.01,21.85


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# config gmaps
gmaps.configure(api_key=g_key)

In [6]:
#identiy data for map
locations = city_df[["LAT", "LNG"]]
humidity = city_df["HUMIDITY"]

#plt
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5

#Display
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

In [10]:
#Narrow down the DataFrame to find your ideal weather condition. For example:
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. 

narrowed_city_df = city_df.loc[(city_df["WIND SPEED"] <= 10) & (city_df["CLOUDINESS"] == 0) & \
                                   (city_df["MAX TEMP"] >= 70) & (city_df["MAX TEMP"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,CITY,CLOUDINESS,COUNTRY,DATE,HUMIDITY,LAT,LNG,MAX TEMP,WIND SPEED
27,27,General Roca,0,AR,1619465076,53,-39.0333,-67.5833,71.60,4.61
114,114,Inhambane,0,MZ,1619465193,84,-23.8650,35.3833,72.59,5.14
168,168,Karratha,0,AU,1619465205,46,-20.7377,116.8463,75.24,7.45
366,366,San Vicente,0,AR,1619465251,34,-26.6167,-54.1333,70.59,4.81
474,474,Rock Sound,0,BS,1619465276,71,24.9000,-76.2000,79.30,7.05
523,523,Sānchi,0,IN,1619465289,27,23.4833,77.7333,78.80,4.61
531,531,San Rafael,0,AR,1619465291,41,-34.6177,-68.3301,72.46,9.82
576,576,General Roca,0,AR,1619465076,53,-39.0333,-67.5833,71.60,4.61
663,663,Inhambane,0,MZ,1619465193,84,-23.8650,35.3833,72.59,5.14
717,717,Karratha,0,AU,1619465205,46,-20.7377,116.8463,75.24,7.45


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# Store into variable named hotel_df.
hotel_df = narrowed_city_df.loc[:,["CITY","COUNTRY", "LAT", "LNG"]]

# add a "Hotel Name" column to the DataFrame.
hotel_df["HOTEL NAME"] = ""


hotel_df

,CITY,COUNTRY,LAT,LNG,HOTEL NAME
27,General Roca,AR,-39.0333,-67.5833,
114,Inhambane,MZ,-23.8650,35.3833,
168,Karratha,AU,-20.7377,116.8463,
366,San Vicente,AR,-26.6167,-54.1333,
474,Rock Sound,BS,24.9000,-76.2000,
523,Sānchi,IN,23.4833,77.7333,
531,San Rafael,AR,-34.6177,-68.3301,
576,General Roca,AR,-39.0333,-67.5833,
663,Inhambane,MZ,-23.8650,35.3833,
717,Karratha,AU,-20.7377,116.8463,


In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["LAT"]
    lng = row["LNG"]
    city_name = row["CITY"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    #time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 27: General Roca.
Closest hotel in General Roca is Estepa Hotel.
------------
Retrieving Results for Index 114: Inhambane.
Closest hotel in Inhambane is Sentidos Beach Retreat.
------------
Retrieving Results for Index 168: Karratha.
Closest hotel in Karratha is Karratha International Hotel.
------------
Retrieving Results for Index 366: San Vicente.
Closest hotel in San Vicente is Hotel Y Restaurante Los Lapachos.
------------
Retrieving Results for Index 474: Rock Sound.
Closest hotel in Rock Sound is Pink Sandy Beaches Villas.
------------
Retrieving Results for Index 523: Sānchi.
Closest hotel in Sānchi is Hotel Sambodhi.
------------
Retrieving Results for Index 531: San Rafael.
Closest hotel in San Rafael is Tower Inn Hotel & Suites Casino Spa Convention Center.
------------
Retrieving Results for Index 576: General Roca.
Closest hotel in General Roca is Estepa Hotel.
------------
Retrieving Results for Index 663: Inhambane.
Closest hotel in Inhambane

In [14]:
hotel_df

,CITY,COUNTRY,LAT,LNG,HOTEL NAME,Hotel Name
27,General Roca,AR,-39.0333,-67.5833,,Estepa Hotel
114,Inhambane,MZ,-23.8650,35.3833,,Sentidos Beach Retreat
168,Karratha,AU,-20.7377,116.8463,,Karratha International Hotel
366,San Vicente,AR,-26.6167,-54.1333,,Hotel Y Restaurante Los Lapachos
474,Rock Sound,BS,24.9000,-76.2000,,Pink Sandy Beaches Villas
523,Sānchi,IN,23.4833,77.7333,,Hotel Sambodhi
531,San Rafael,AR,-34.6177,-68.3301,,Tower Inn Hotel & Suites Casino Spa Convention...
576,General Roca,AR,-39.0333,-67.5833,,Estepa Hotel
663,Inhambane,MZ,-23.8650,35.3833,,Sentidos Beach Retreat
717,Karratha,AU,-20.7377,116.8463,,Karratha International Hotel


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{CITY}</dd>
<dt>Country</dt><dd>{COUNTRY}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["LAT", "LNG"]]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))